# Table of Contents



1. Reference Python Code for Selenium + BeautifulSoup while working with API


2. Selenium for extracting data from website


3. Scraping financial website: Estimize


4. Attempt to scrap LinkedIn


In [2]:
!pip install selenium

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)


# 1. Reference Python Code for Selenium + BeautifulSoup while working with API

In [ ]:
import bs4 as bs
import urllib.request
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from datetime import datetime
import re
import time
import math
import csv
import shlex
import pandas as pd
from datetime import datetime
import random

In [3]:
#open the page
#driver = webdriver.Chrome("E:/Data Science/chromedriver_win32 (2)/chromedriver.exe")
driver = webdriver.Firefox(executable_path=r'C:/Users/anshu/Downloads/geckodriver.exe')

### Some useful functions

In [1]:
# function to convert to readable time
def get_time(time):
    ts = int(time)
    # if you encounter a "year is out of range" error the timestamp
    # may be in milliseconds, try `ts /= 1000` in that case
    return (datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

def random_wait(tm=5):
    t=(1+random.randint(0, tm-1))
    time.sleep(t)

In [ ]:
'''
Declare initial parameters
'''
end_time_int = 1684641599
start_time_int = 1367121600
duration = 604799
end = end_time_int
res_df = pd.DataFrame()
itr=1



'''
Open the browser
'''
driver = webdriver.Firefox(executable_path=r'C:/Users/anshu/Downloads/geckodriver.exe')



'''
Main Code
'''
while end>=start_time_int:
    print("itr: ",itr)
    '''
    Get the start date of each period
    '''
    start = end - duration
    
    
    
    '''
    Extract Data for the designated duration
    '''
    ## if no data is obtained, break the loop ##
    # api link
    link = "https://api.coinmarketcap.com/data-api/v3/cryptocurrency/detail/chart?id=1&range={}~{}".format(start,end)
    
    # open webpage
    driver.get(link)
    random_wait(3)

    # click to expand
    driver.find_element_by_css_selector("button[class='btn expand']").click()
    random_wait(2)
    
    # get the soup
    soup = bs.BeautifulSoup(driver.page_source, "html.parser")
    
    # get the time rows
    timestamp_list = soup.find_all("tr",{'aria-level':'3'})
    
    # if no data is obtained, break the loop
    if len(timestamp_list)==0:
        break
    
    
    
    '''
    Convert to DataFrame
    '''
    res_list = []
    for timestamp in timestamp_list:

        # try to extract without beautiful soup
        time_str = timestamp.text
        link_0 = "/data/points/{}/c/0".format(time_str)
        link_1 = "/data/points/{}/c/1".format(time_str)
        link_2 = "/data/points/{}/c/2".format(time_str)
        price = driver.find_element_by_css_selector("tr[id='{}']".format(link_0)).text[2:]
        vol = driver.find_element_by_css_selector("tr[id='{}']".format(link_1)).text[2:]
        mcap = driver.find_element_by_css_selector("tr[id='{}']".format(link_2)).text[2:]
        
        # store it in dataframe
        res_list.append({'time':time_str,
                         'price':price,
                         'vol_past24hr':vol,
                         'mcap':mcap})


    tmp_df = pd.DataFrame(res_list)
    tmp_df['time_new'] = tmp_df.apply(lambda x: get_time(x['time']),axis=1)    
    res_df = pd.concat([res_df,tmp_df])
    
    
    
    '''
    Update the end date for next duration
    '''
    end = start-1
    itr+=1

### Inefficient with beautifulsoup

In [ ]:
# '''
# Declare initial parameters
# '''
# end_time_int = 1684641599
# start_time_int = 1678593609  # 1367121600
# duration = 604799
# end = end_time_int
# res_df = pd.DataFrame()
# itr=1



# '''
# Open the browser
# '''
# driver = webdriver.Firefox(executable_path=r'C:/Users/anshu/Downloads/geckodriver.exe')



# '''
# Main Code
# '''
# while end>=start_time_int:
#     print("itr: ",itr)
#     '''
#     Get the start date of each period
#     '''
#     start = end - duration
    
    
    
#     '''
#     Extract Data for the designated duration
#     '''
#     ## if no data is obtained, break the loop ##
#     # api link
#     link = "https://api.coinmarketcap.com/data-api/v3/cryptocurrency/detail/chart?id=1&range={}~{}".format(start,end)
    
#     # open webpage
#     driver.get(link)
#     random_wait(3)

#     # click to expand
#     driver.find_element_by_css_selector("button[class='btn expand']").click()
#     random_wait(2)
    
#     # get the soup
#     soup = bs.BeautifulSoup(driver.page_source, "html.parser")
    
#     # get the time rows
#     timestamp_list = soup.find_all("tr",{'aria-level':'3'})
    
#     # if no data is obtained, break the loop
#     if len(timestamp_list)==0:
#         break
    
    
    
#     '''
#     Convert to DataFrame
#     '''
#     res_list = []
#     for timestamp in timestamp_list:

#         # get timestamp
#         time_str = timestamp.text

#         # get c elements
#         c_list = soup.find_all("tr",id = re.compile("/data/points/{}/c/".format(time_str)))

#         # store it in dataframe
#         res_list.append({'time':time_str,
#                          'price':c_list[0].find("span",class_='objectBox objectBox-number').text,
#                          'vol_past24hr':c_list[1].find("span",class_='objectBox objectBox-number').text,
#                          'mcap':c_list[2].find("span",class_='objectBox objectBox-number').text})


#     tmp_df = pd.DataFrame(res_list)
#     tmp_df['time_new'] = tmp_df.apply(lambda x: get_time(x['time']),axis=1)    
#     res_df = pd.concat([res_df,tmp_df])
    
    
    
#     '''
#     Update the end date for next duration
#     '''
#     end = start-1
#     itr+=1

# 2. Selenium for extracting data from website

In [65]:
#open the website page
driver = webdriver.Firefox()
driver.get("https://www.basspro.com/shop/en/guns")

In [68]:
# Get guns category
# gtype = driver.find_element_by_id("categoryFacetList_5_-2009_3074457345618347091")
# time.sleep(10)
# options = gtype.find_elements_by_tag_name("li")
options = ['https://www.basspro.com/shop/en/centerfire-pistol','https://www.basspro.com/shop/en/shotgun',
          'https://www.basspro.com/shop/en/centerfire-rifles','https://www.basspro.com/shop/en/rimfire-rifle',
          'https://www.basspro.com/shop/en/rimfire-pistols']
data = []
for option in options:
#     try:
#         option.get_attribute("href").click()
#         print('click1')
#         time.sleep(5)
#     except:
#         try: 
#             driver.get(option.get_attribute("href"))
#             print('click2')
#             time.sleep(5)
        
#         except:
#             time.sleep(5)
    driver.get(option)
    print('clicked!')
    time.sleep(5)    
#     page = driver.find_element_by_id("dijit__WidgetBase_0")
    i=0
    it=1
    while(i<2):
        page = driver.find_element_by_css_selector("div [class='product_listing_container']")
        guns = page.find_elements_by_tag_name("li")
        print("PAGE NO. ",it)
        try:
            if not guns:
                i=1000
        except:
            print("DATA LEFT")
            continue
        for gun in guns:
            name = gun.find_element_by_css_selector("div[class='product_name']").text
            price = gun.find_element_by_css_selector("span[itemprop='price']").get_attribute("content")
            print(name,price)
            data.append({'name':name,'price':price})
        next_pg = driver.find_element_by_css_selector("i[alt='Show next page'][class='icon icon-arrow-right']")
        next_pg.click()
        time.sleep(5)
        it=it+1
        

In [67]:
import csv
keys = data[0].keys()
with open('guns.csv', 'w', newline='')  as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(data)

In [ ]:
# <i [alt='Show next page'][class='icon icon-arrow-right'] 
# id="WC_SearchBasedNavigationResults_pagination_link_right_categoryResults_top_img"></i>

In [ ]:
# <i alt="Show next page" class="icon icon-arrow-right" 
# id="WC_SearchBasedNavigationResults_pagination_link_right_categoryResults_top_img"></i>

# 3. Scraping financial website: Estimize

In [ ]:
# 1. Import packages
import pandas as pd
import numpy as np
import re
from fake_useragent import UserAgent
from selenium.webdriver.chrome.options import Options

import time
import os
import bs4 as bs
import urllib.request
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

from lxml.html import fromstring
import requests
from itertools import cycle
import traceback
from selenium.webdriver.common.proxy import Proxy, ProxyType
import re
import random
import pyautogui as py

from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# from get_data_each_stock import *


# 2. Open browser
def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:10]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            #Grabbing IP and corresponding PORT
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies


    ### create config from proxy
def create_proxy_config(proxy):
    myProxy = proxy
    proxy = Proxy({
        'proxyType': ProxyType.MANUAL,
        'httpProxy': myProxy,
        'ftpProxy': myProxy,
        'sslProxy': myProxy,
        'noProxy': '' # set this value as desired
        })
    return proxy

proxies = get_proxies()
proxies = get_proxies()
if len(proxies) ==0:
    proxy = None
    proxy_pool = None
else:
    proxy_pool = cycle(proxies)
    proxy = next(proxy_pool)
    proxy = create_proxy_config(proxy)

def open_browser(proxy,proxy_pool):
    try:
        driver = webdriver.Firefox(executable_path=r'C:/Users/anshu/Downloads/geckodriver.exe',proxy=proxy)
    except:
        if len(proxy_pool)==0:
            proxy=None
        else:
            proxy = next(proxy_pool)
            proxy = create_proxy_config(proxy)
            driver = webdriver.Firefox(executable_path=r'C:/Users/anshu/Downloads/geckodriver.exe',proxy=proxy)
    time.sleep(2)
    return driver,proxy,proxy_pool



# initialise stock list
stock_list = []

# get drivers
driver, proxy,proxy_pool = open_browser(proxy,proxy_pool)
# driver.get('https://www.estimize.com/sectors/consumer-discretionary')

# # click to get all stocks list
# driver.find_element_by_css_selector('div[class="pagination-footer').click()
# time.sleep(2)

# # append list of links
# elm_list = []
# for elm in driver.find_elements_by_css_selector('a[class="linked-row closed'):
#     elm = elm.get_attribute('href')
#     elm = (elm[:-9])
#     elm_list.append(elm)

# if len(elm_list)==0:
#     print("using opened one now")
#     for elm in driver.find_elements_by_css_selector('a[class="linked-row opened'):
#         elm = elm.get_attribute('href')
#         elm = (elm[:-9])
#         elm_list.append(elm) 

# # click href link    
# for elm in elm_list:    
#     print(elm)
#     try:
#         stock_list = get_data_from_webpage_2(driver,elm,stock_list)
#     except:
#         continue

# # save the stock_list as df
# tmp_df = pd.DataFrame(stock_list)
# tmp_df.to_csv('../output/stock_list_2.csv') 


def get_data_from_webpage_2(driver,elm,stock_list):    
    
    # get the webpage
    driver.get(elm)
    time.sleep(2)
    
    #click to get table
    try:
        driver.find_element_by_css_selector('a[class="estimate-bar__button"]').click()
        time.sleep(2)
    except:
        print("okay")
    
    # get the name of result sheet
    name = driver.find_element_by_css_selector('h1[class="release-header-information-title"]').text
    name = name.replace(" ","")
    
    # get the next earning dates
    date_elm = driver.find_element_by_css_selector('span[class="multiform__countdown-reports-at"]').text
    day = date_elm.split()[1]
    date = date_elm.split()[2]
    timings = date_elm.split()[3]
    print(day,date,timings)
    
    # save the name of company and link in the stock_list
    stock_list.append({'name':name, 'link':elm, 'earn_day':day, 'earn_date':date, 'earn_time':timings})
    
    # create dataframe
    res_df = pd.DataFrame()
    
    # get column names from top row
    row = driver.find_element_by_css_selector('tr[class="multiform-tr-qtrs"]')
    cols = row.find_elements_by_css_selector('td[class="multiform-td-qtr"]')
    col_names = []
    for col in cols:
        col_names.append(col.text)
    
    # get estimize value and count columns
    row = driver.find_element_by_css_selector('tr[class="multiform-tr-consensus multiform-tr-estimize"]')
    cols = row.find_elements_by_css_selector('td[class="multiform-td-data"]')
    est_val_dict = {}
    est_cnt_dict = {}
    i=0
    for col in cols:
        tmp = col.text.split()
        try:
            est_val_dict[col_names[i]] = tmp[0]
        except:
            est_val_dict[col_names[i]] = np.nan
        try:
            est_cnt_dict[col_names[i]] = tmp[1]
        except:
            est_cnt_dict[col_names[i]] = np.nan
        i=i+1
    est_val_df = pd.DataFrame([est_val_dict])
    est_cnt_df = pd.DataFrame([est_cnt_dict])
    res_df = pd.concat([res_df,est_val_df],ignore_index=True)
    res_df = pd.concat([res_df,est_cnt_df],ignore_index=True)
    
    # get wall street columns
    row = driver.find_element_by_css_selector('tr[class="multiform-tr-consensus multiform-tr-wallstreet"]')
    cols = row.find_elements_by_css_selector('td[class="multiform-td-data"]')
    wall_st_dict = {}
    i=0
    for col in cols:
        wall_st_dict[col_names[i]] = col.text
        i=i+1
    wall_st_df = pd.DataFrame([wall_st_dict])
    res_df = pd.concat([res_df,wall_st_df],ignore_index=True)
    
    # print("res: ",res_df)
    
    # get actual values columns
    row = driver.find_element_by_css_selector('tr[class="multiform-tr-actuals"]')
    cols = row.find_elements_by_css_selector('td[class="multiform-td-data"]')
    act_val_dict = {}
    act_gth_dict = {}
    i=0
    for col in cols:
        tmp = col.text.split()
        act_val_dict[col_names[i]] = tmp[0]
        act_gth_dict[col_names[i]] = tmp[1]
        i=i+1
    act_val_df = pd.DataFrame([act_val_dict])
    res_df = pd.concat([res_df, act_val_df],ignore_index=True)
    act_gth_df = pd.DataFrame([act_gth_dict])
    res_df = pd.concat([res_df, act_gth_df],ignore_index=True)
    
    # quit driver
    # driver.close()
    # print("res: ",res_df)
    
    # save dataframe
    res_df.to_csv('../output/' + name + '.csv')
    
    return stock_list


def get_est_file_stock_list(driver,proxy,proxy_pool,ind_link = 'https://www.estimize.com/sectors/consumer-discretionary',op_file='stock_list_2.csv'):
    # initiate stock_list
    stock_list = []
    
    # open Consumer Durable webpage  
    driver, proxy,proxy_pool = open_browser(proxy,proxy_pool)
    driver.get(ind_link)
    
    # click to get all stocks list
    driver.find_element_by_css_selector('div[class="pagination-footer').click()
    time.sleep(2)
    
    # append list of links
    elm_list = []
    for elm in driver.find_elements_by_css_selector('a[class="linked-row closed]'):
        elm = elm.get_attribute('href')
        elm = (elm[:-9])
        elm_list.append(elm)
        
    # for elm in driver.find_elements_by_css_selector('a[class="linked-row opened]'):
    #     elm = elm.get_attribute('href')
    #     elm = (elm[:-9])
    #     elm_list.append(elm)        
    
    if len(elm_list)==0:
        print("using opened one now")
        for elm in driver.find_elements_by_css_selector('a[class="linked-row opened'):
            elm = elm.get_attribute('href')
            elm = (elm[:-9])
            elm_list.append(elm) 
    
    # click href link    
    for elm in elm_list:    
        print(elm)
        try:
            stock_list = get_data_from_webpage_2(driver,elm,stock_list)
        except:
            continue
    
    # save the stock_list as df
    try:
        res_df = pd.read_csv('../output/'+op_file)
    except:
        res_df = pd.DataFrame()
    tmp_df = pd.DataFrame(stock_list)
    res_df = pd.concat([res_df,tmp_df],)
    res_df.to_csv('../output/' + op_file)
    return    

# consumer discretionary
lnk = 'https://www.estimize.com/sectors/consumer-discretionary'
get_est_file_stock_list(driver,proxy,proxy_pool,ind_link = lnk,op_file='stock_list_2.csv')

# consumer staple
lnk = 'https://www.estimize.com/sectors/consumer-staples'
get_est_file_stock_list(driver,proxy,proxy_pool,ind_link = lnk,op_file='stock_list_2.csv')

# energy
lnk = 'https://www.estimize.com/sectors/energy'
get_est_file_stock_list(driver,proxy,proxy_pool,ind_link = lnk,op_file='stock_list_2.csv')

# financials
lnk = 'https://www.estimize.com/sectors/financials'
get_est_file_stock_list(driver,proxy,proxy_pool,ind_link = lnk,op_file='stock_list_2.csv')

# healthcare
lnk = 'https://www.estimize.com/sectors/health-care'
get_est_file_stock_list(driver,proxy,proxy_pool,ind_link = lnk,op_file='stock_list_2.csv')

# industrials
lnk = 'https://www.estimize.com/sectors/industrials'
get_est_file_stock_list(driver,proxy,proxy_pool,ind_link = lnk,op_file='stock_list_2.csv')

# it
lnk = 'https://www.estimize.com/sectors/information-technology'
get_est_file_stock_list(driver,proxy,proxy_pool,ind_link = lnk,op_file='stock_list_2.csv')

# materials
lnk = 'https://www.estimize.com/sectors/materials'
get_est_file_stock_list(driver,proxy,proxy_pool,ind_link = lnk,op_file='stock_list_2.csv')

# telecom
lnk = 'https://www.estimize.com/sectors/telecommunication-services'
get_est_file_stock_list(driver,proxy,proxy_pool,ind_link = lnk,op_file='stock_list_2.csv')

# utilities
lnk = 'https://www.estimize.com/sectors/utilities'
get_est_file_stock_list(driver,proxy,proxy_pool,ind_link = lnk,op_file='stock_list_2.csv')

# 4. Attempt to scrap LinkedIn

* Includes attempt to mask the scrapping activity
* Random time, clicks and hovering

In [2]:
import time, random, os, csv, platform
import logging
from selenium import webdriver
# from future import annotations
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import pyautogui
import random

from urllib.request import urlopen
from webdriver_manager.chrome import ChromeDriverManager
import re
import yaml
from datetime import datetime, timedelta
from selenium.webdriver.common.action_chains import ActionChains

log = logging.getLogger(__name__)


In [3]:
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--ignore-certificate-errors")
options.add_argument('--no-sandbox')
options.add_argument("--disable-extensions")

# Disable webdriver flags or you will be easily detectable
options.add_argument("--disable-blink-features")
options.add_argument("--disable-blink-features=AutomationControlled")

In [4]:
browser = webdriver.Chrome(ChromeDriverManager().install())
browser.get("https://www.linkedin.com/login?trk=guest_homepage-basic_nav-header-signin")

# username = #fill here
# password = #fill here

[WDM] - Downloading:  95%|█████████████████████████████████████████████████████▎  | 6.48M/6.81M [00:00<00:00, 29.4MB/s]<ipython-input-4-b480e122d08e>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())
[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.81M/6.81M [00:20<00:00, 29.4MB/s]

In [ ]:
user_field = browser.find_element("id","username")
pw_field = browser.find_element("id","password")
login_button = browser.find_element("xpath",
            '//*[@id="organic-div"]/form/div[3]/button')
user_field.send_keys(username)
user_field.send_keys(Keys.TAB)
time.sleep(2)
pw_field.send_keys(password)
time.sleep(2)
login_button.click()
time.sleep(3)

In [ ]:
browser.get("https://www.linkedin.com/jobs/search/?currentJobId=3537172049&f_TPR=r86400&keywords=product%20management%20intern")

In [ ]:
res_list = []

In [ ]:
i = 1

In [ ]:
# wait
t = (random.randint(1,10))
time.sleep(t)

In [ ]:
# get page
page = browser.find_elements("xpath",'//main[@id="main"]')
# browser.execute_script("window.stop();")

In [ ]:
# get elements on page
max_search = 3000
for i in range(101,max_search):
    try:
        txt = page[0].find_elements("xpath",
                                '//li[@id="ember{}"]'.format(i))[0].text
        res_list.append(txt)
#         max_search=i+200
    except:
        continue

In [ ]:
  
# increase page number
i+=1
res_list

In [ ]:
try:
    next_pg = browser.find_element("xpath",'//button[@aria-label="Page {}"]'.format(9))
    t = (random.randint(1,20))
    time.sleep(t)
except:
    print("ded")
next_pg.click()

In [ ]:
browser.refresh()

## Newcode

In [5]:
def wait_():
    # wait
    t = (random.randint(1,5))
    time.sleep(t)
    
def hover_(el,driver=browser):
    action = ActionChains(driver)
#     el = browser.find_elements("xpath",'//main[@id="main"]')[0]
    action.move_to_element_with_offset(el, random.randint(1,20), random.randint(1,20))
    action.click()
    action.perform()

In [ ]:
browser = webdriver.Chrome(ChromeDriverManager().install())

In [ ]:
browser.get("https://www.linkedin.com/login?trk=guest_homepage-basic_nav-header-signin")

# username = 'fillhere@gmail.com'
# password = "*Pnp368#68456845"
wait_()
user_field = browser.find_element("id","username")
pw_field = browser.find_element("id","password")
el = user_field
wait_()

login_button = browser.find_element("xpath",
            '//*[@id="organic-div"]/form/div[3]/button')
user_field.send_keys(username)
user_field.send_keys(Keys.TAB)
wait_()
pw_field.send_keys(password)
wait_()
login_button.click()
wait_()

In [ ]:
browser.get("https://www.linkedin.com/jobs/search/?currentJobId=3537172049&f_TPR=r86400&keywords=product%20management%20intern")

In [ ]:
res_list = []
res = []
itr=1


for i in range(itr-1,100):
    # refresh page
    browser.refresh()
    wait_()
    
    # increase iterator
    itr+=1
    
    # get page
    time.sleep(20)
    page = browser.find_elements("xpath",'//main[@id="main"]')
    wait_()
#     browser.execute_script("window.stop();")


    for elm in page[0].text.split("Close")[0].split("Job alert")[1].replace("On-site","||").replace("Actively recruiting","||").replace("Promoted","||").replace("Within the past 24 hours","||").replace("ago","||").replace("Easy Apply","||").split("||"):
        if (("summer" in elm.lower()) | ("intern" in elm.lower())):
            res_list.append(elm)
            for row in res_list:
                res.append(row.split("\n"))

    try:
        next_pg = browser.find_element("xpath",'//button[@aria-label="Page {}"]'.format(itr))
        wait_()
    except:
        print("ded")
        # refresh page
#         browser.get("https://www.linkedin.com/jobs/search/?currentJobId=3537172049&f_TPR=r86400&keywords=product%20management%20intern")
        browser.refresh()
        wait_()
        itr-=1
        
    next_pg.click()

In [ ]:
res_list